# increase gpu memory


If you face disk quota exceeded, try running this cell.  

In [ ]:
import os
import tensorflow as tf
from transformers.utils import logging

# Limit GPU memory usage
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

# Enable memory growth for TensorFlow
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2025-02-23 10:12:02.326321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740301922.361068  714330 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740301922.370638  714330 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Finetune on FashionPedia  (frozone resnet backbone)

For this experiment we refered : https://huggingface.co/learn/cookbook/fine_tuning_detr_custom_dataset

Download dataset: https://huggingface.co/datasets/detection-datasets/fashionpedia

NOTE: GenAI tools were used for debugging and codinf assistance.

1. Load Dataset

In [ ]:
from datasets import load_dataset
import numpy as np
from PIL import Image, ImageDraw
import io
import torch
import albumentations as A
from transformers import AutoImageProcessor
from transformers import AutoModelForObjectDetection
import matplotlib.pyplot as plt
import numpy as np
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.nn.functional import softmax
from transformers import TrainingArguments
from transformers import Trainer
from prettytable import PrettyTable
from transformers import pipeline
import numpy as np
from PIL import Image, ImageDraw
import numpy as np
from transformers import pipeline
import numpy as np
import io
import os
import random





In [ ]:
dataset = load_dataset("parquet", data_dir="/home/utn/firi22ka/Desktop/jenga/mlp/g10/fashionpedia/data", cache_dir="/home/utn/firi22ka/Desktop/jenga/hf_cache")

In [ ]:
dataset["train"][0]

In [3]:
train_dataset = dataset["train"]
test_dataset = dataset["validation"]

In [4]:
# Uncomment below code  to select  10,000 imagee for training and 1000 images for testing
# train_subset = dataset["train"].select(range(10000))
test_subset = dataset["validation"].select(range(1000))

In [ ]:


# converting id to lables
id2label = {
    0: "shirt, blouse",
    1: "top, t-shirt, sweatshirt",
    2: "sweater",
    3: "cardigan",
    4: "jacket",
    5: "vest",
    6: "pants",
    7: "shorts",
    8: "skirt",
    9: "coat",
    10: "dress",
    11: "jumpsuit",
    12: "cape",
    13: "glasses",
    14: "hat",
    15: "headband, head covering, hair accessory",
    16: "tie",
    17: "glove",
    18: "watch",
    19: "belt",
    20: "leg warmer",
    21: "tights, stockings",
    22: "sock",
    23: "shoe",
    24: "bag, wallet",
    25: "scarf",
    26: "umbrella",
    27: "hood",
    28: "collar",
    29: "lapel",
    30: "epaulette",
    31: "sleeve",
    32: "pocket",
    33: "neckline",
    34: "buckle",
    35: "zipper",
    36: "applique",
    37: "bead",
    38: "bow",
    39: "flower",
    40: "fringe",
    41: "ribbon",
    42: "rivet",
    43: "ruffle",
    44: "sequin",
    45: "tassel",
}


label2id = {v: k for k, v in id2label.items()}

Visualizing a sample image from the dataset

In [ ]:

def draw_image_from_idx(dataset, idx):
    sample = dataset[idx]
    image = sample["image"]
    image_bytes = image["bytes"]  # Extract bytes from dictionary
    image = Image.open(io.BytesIO(image_bytes))
    # print(image)  
    annotations = sample["objects"]
    draw = ImageDraw.Draw(image)
    width, height = sample["width"], sample["height"]

    print(annotations)

    for i in range(len(annotations["bbox_id"])):
        box = annotations["bbox"][i]
        x1, y1, x2, y2 = tuple(box)
        draw.rectangle((x1, y1, x2, y2), outline="red", width=3)
        draw.text((x1, y1), id2label[annotations["category"][i]], fill="green")

    return image

 
# draw_image_from_idx(dataset=test_dataset, idx=200)  # You can test changing this id

Data Filtering: removing invalid bounding boxes 

In [7]:
def filter_invalid_bboxes(example):
    valid_bboxes = []
    valid_bbox_ids = []
    valid_categories = []
    valid_areas = []

    for i, bbox in enumerate(example["objects"]["bbox"]):
        x_min, y_min, x_max, y_max = bbox[:4]
        if x_min < x_max and y_min < y_max:
            valid_bboxes.append(bbox)
            valid_bbox_ids.append(example["objects"]["bbox_id"][i])
            valid_categories.append(example["objects"]["category"][i])
            valid_areas.append(example["objects"]["area"][i])
        else:
            print(
                f"Image with invalid bbox: {example['image_id']} Invalid bbox detected and discarded: {bbox} - bbox_id: {example['objects']['bbox_id'][i]} - category: {example['objects']['category'][i]}"
            )

    example["objects"]["bbox"] = valid_bboxes
    example["objects"]["bbox_id"] = valid_bbox_ids
    example["objects"]["category"] = valid_categories
    example["objects"]["area"] = valid_areas

    return example


train_dataset = train_dataset.map(filter_invalid_bboxes)
# test_dataset = test_dataset.map(filter_invalid_bboxes)
test_dataset = test_subset.map(filter_invalid_bboxes) #test on 1000 test images

In [8]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['image_id', 'image', 'width', 'height', 'objects'],
    num_rows: 45623
})
Dataset({
    features: ['image_id', 'image', 'width', 'height', 'objects'],
    num_rows: 1000
})


Data Augmentation: Applying data augmentation techniques for better data quality

In [ ]:

train_transform = A.Compose(
    [
        A.LongestMaxSize(500),
        A.PadIfNeeded(500, 500, border_mode=0, value=(0, 0, 0)),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
        A.Rotate(limit=10, p=0.5),
        A.RandomScale(scale_limit=0.2, p=0.5),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
    ],
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=["category"]),
)

val_transform = A.Compose(
    [
        A.LongestMaxSize(500),
        A.PadIfNeeded(500, 500, border_mode=0, value=(0, 0, 0)),
    ],
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=["category"]),
)

/tmp/ipykernel_714330/2247774584.py:6: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(500, 500, border_mode=0, value=(0, 0, 0)),
/tmp/ipykernel_714330/2247774584.py:21: UserWarning: Argument(s) 'value' are not valid for transform PadIfNeeded
  A.PadIfNeeded(500, 500, border_mode=0, value=(0, 0, 0)),


Loading pretrained weights 

In [ ]:
# uncomment the models you want to train on or test on , and comment out other current chekpoint
# checkpoint = "/home/utn/firi22ka/Desktop/jenga/mlp/g10/test/detr-resnet-50-fashionpedia-full-finetuned/checkpoint-5000" 
# checkpoint = '/home/utn/firi22ka/Desktop/jenga/mlp/g10/test/detr-resnet-50-fashionpedia-full-finetuned_10000/checkpoint-12500'
checkpoint = '/home/utn/firi22ka/Desktop/jenga/mlp/g10/test/detr-resnet-50-fashionpedia-full-finetuned_5epochs/checkpoint-25000'
# checkpoint = "facebook/detr-resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

Converting data into DeRT compatible fromat for training 

In [11]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):
        new_ann = {
            "image_id": image_id,
            "category_id": category[i],
            "isCrowd": 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }
        annotations.append(new_ann)

    return annotations


def convert_voc_to_coco(bbox):
    xmin, ymin, xmax, ymax = bbox
    width = xmax - xmin
    height = ymax - ymin
    return [xmin, ymin, width, height]


def transform_aug_ann(examples, transform):
    image_ids = examples["image_id"]
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples["image"], examples["objects"]):
        if isinstance(image, dict) and "bytes" in image:
            image = Image.open(io.BytesIO(image["bytes"]))
        
        image = np.array(image.convert("RGB"))[:, :, ::-1]
        out = transform(image=image, bboxes=objects["bbox"], category=objects["category"])

        area.append(objects["area"])
        images.append(out["image"])

        # Convert to COCO format
        converted_bboxes = [convert_voc_to_coco(bbox) for bbox in out["bboxes"]]
        bboxes.append(converted_bboxes)

        categories.append(out["category"])

    targets = [
        {"image_id": id_, "annotations": formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]

    return image_processor(images=images, annotations=targets, return_tensors="pt")


def transform_train(examples):
    return transform_aug_ann(examples, transform=train_transform)


def transform_val(examples):
    return transform_aug_ann(examples, transform=val_transform)


train_dataset_transformed = train_dataset.with_transform(transform_train)
test_dataset_transformed = test_dataset.with_transform(transform_val)

In [ ]:

# Updated draw function to accept an optional transform
def draw_augmented_image_from_idx(dataset, idx, transform=None):
    sample = dataset[idx]
    image = sample["image"]
    image_bytes = image["bytes"]  # Extract bytes from dictionary
    image = Image.open(io.BytesIO(image_bytes))
    annotations = sample["objects"]

    # Convert image to RGB and NumPy array
    image = np.array(image.convert("RGB"))[:, :, ::-1]

    if transform:
        augmented = transform(image=image, bboxes=annotations["bbox"], category=annotations["category"])
        image = augmented["image"]
        annotations["bbox"] = augmented["bboxes"]
        annotations["category"] = augmented["category"]

    image = Image.fromarray(image[:, :, ::-1])  # Convert back to PIL Image
    draw = ImageDraw.Draw(image)
    width, height = sample["width"], sample["height"]

    for i in range(len(annotations["bbox_id"])):
        box = annotations["bbox"][i]
        x1, y1, x2, y2 = tuple(box)

        # Normalizing  coordinates
        if max(box) <= 1.0:
            x1, y1 = int(x1 * width), int(y1 * height)
            x2, y2 = int(x2 * width), int(y2 * height)
        else:
            x1, y1 = int(x1), int(y1)
            x2, y2 = int(x2), int(y2)

        draw.rectangle((x1, y1, x2, y2), outline="red", width=3)
        draw.text((x1, y1), id2label[annotations["category"][i]], fill="green")

    return image


def plot_augmented_images(dataset, indices, transform=None):
    """
    Plot images and their annotations with optional augmentation.
    """
    num_rows = len(indices) // 3
    num_cols = 3
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))

    for i, idx in enumerate(indices):
        row = i // num_cols
        col = i % num_cols

        # Draw augmented image
        image = draw_augmented_image_from_idx(dataset, idx, transform=transform)

        # Display image on the corresponding subplot
        axes[row, col].imshow(image)
        axes[row, col].axis("off")

    plt.tight_layout()
    plt.show()


# plotting augmented images
# plot_augmented_images(train_dataset, range(9), transform=train_transform)

# Fine-tuning DeRT on complete dataset

Initializing model and save directory for fintuned models

In [ ]:

model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

output_dir = "detr-resnet-50-fashionpedia-full-finetuned_5epochs"  # edit this to save fine-tuned model in your choice of directory.

Uncomment below cell and tun if you want to finetune dert by freezing backbone

In [ ]:

# model = AutoModelForObjectDetection.from_pretrained(
#     checkpoint,
#     id2label=id2label,
#     label2id=label2id,
#     ignore_mismatched_sizes=True,
# )
# for name, param in model.named_parameters():
#     if "backbone" in name:
#         param.requires_grad = False
#         print(f"Frozen parameter: {name}")

# # Print trainable parameters
# print("\nTrainable parameters:")
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.shape}")
# output_dir = "detr_finetuned_coco_backobone_freeze_100"  # edit this to save fine-tuned model in your choice of directory.

Defining  training configurations

In [ ]:

# Define the training arguments

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    fp16=True,
    save_steps=100,
    logging_steps=1,
    learning_rate=1e-5,
    weight_decay=1e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    evaluation_strategy="steps",
    eval_steps=50,
    eval_strategy="steps",
    batch_eval_metrics=True,
)

/home/utn/firi22ka/Desktop/jenga/mlp/g10/vit_env/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

def denormalize_boxes(boxes, width, height):
    boxes = boxes.clone()
    boxes[:, 0] *= width  # xmin
    boxes[:, 1] *= height  # ymin
    boxes[:, 2] *= width  # xmax
    boxes[:, 3] *= height  # ymax
    return boxes


batch_metrics = []


def compute_metrics(eval_pred, compute_result):
    global batch_metrics

    (loss_dict, scores, pred_boxes, last_hidden_state, encoder_last_hidden_state), labels = eval_pred

    image_sizes = []
    target = []
    for label in labels:

        image_sizes.append(label["orig_size"])
        width, height = label["orig_size"]
        denormalized_boxes = denormalize_boxes(label["boxes"], width, height)
        target.append(
            {
                "boxes": denormalized_boxes,
                "labels": label["class_labels"],
            }
        )
    predictions = []
    for score, box, target_sizes in zip(scores, pred_boxes, image_sizes):
        # Extract the bounding boxes, labels, and scores from the model's output
        pred_scores = score[:, :-1]  # Exclude the no-object class
        pred_scores = softmax(pred_scores, dim=-1)
        width, height = target_sizes
        pred_boxes = denormalize_boxes(box, width, height)
        pred_labels = torch.argmax(pred_scores, dim=-1)

        # Get the scores corresponding to the predicted labels
        pred_scores_for_labels = torch.gather(pred_scores, 1, pred_labels.unsqueeze(-1)).squeeze(-1)
        predictions.append(
            {
                "boxes": pred_boxes,
                "scores": pred_scores_for_labels,
                "labels": pred_labels,
            }
        )

    metric = MeanAveragePrecision(box_format="xywh", class_metrics=True)

    if not compute_result:
        # Accumulate batch-level metrics
        batch_metrics.append({"preds": predictions, "target": target})
        return {}
    else:
        # Compute final aggregated metrics
        # Aggregate batch-level metrics (this should be done based on your metric library's needs)
        all_preds = []
        all_targets = []
        for batch in batch_metrics:
            all_preds.extend(batch["preds"])
            all_targets.extend(batch["target"])

        # Update metric with all accumulated predictions and targets
        metric.update(preds=all_preds, target=all_targets)
        metrics = metric.compute()

        # Convert and format metrics as needed
        classes = metrics.pop("classes")
        map_per_class = metrics.pop("map_per_class")
        mar_100_per_class = metrics.pop("mar_100_per_class")

        for class_id, class_map, class_mar in zip(classes, map_per_class, mar_100_per_class):
            class_name = id2label[class_id.item()] if id2label is not None else class_id.item()
            metrics[f"map_{class_name}"] = class_map
            metrics[f"mar_100_{class_name}"] = class_mar

        # Round metrics for cleaner output
        metrics = {k: round(v.item(), 4) for k, v in metrics.items()}

        # Clear batch metrics for next evaluation
        batch_metrics = []

        return metrics

In [16]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]

    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = labels

    return batch

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=train_dataset_transformed,
    eval_dataset=test_dataset_transformed,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)
# uncomment below line to start training using pretrained weights
# trainer.train()

#uncomment below line to start training from a fine-tuned checkpoint
# trainer.train(resume_from_checkpoint= "/home/utn/firi22ka/Desktop/jenga/mlp/g10/test/detr-resnet-50-fashionpedia-full-finetuned_5epochs/checkpoint-16400")

/tmp/ipykernel_714330/3435597507.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# uncomment this if you fet disk quota exceeded error. 
# !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Evaluate your fine-tuned model on test dataset

In [24]:
metrics = trainer.evaluate(test_dataset_transformed)
print(metrics)

{'eval_loss': 1.9507602453231812, 'eval_model_preparation_time': 0.0025, 'eval_map': 0.0436, 'eval_map_50': 0.0812, 'eval_map_75': 0.0419, 'eval_map_small': 0.0167, 'eval_map_medium': 0.0464, 'eval_map_large': 0.0732, 'eval_mar_1': 0.0894, 'eval_mar_10': 0.1299, 'eval_mar_100': 0.1315, 'eval_mar_small': 0.0458, 'eval_mar_medium': 0.1432, 'eval_mar_large': 0.166, 'eval_map_shirt, blouse': 0.0069, 'eval_mar_100_shirt, blouse': 0.0368, 'eval_map_top, t-shirt, sweatshirt': 0.0876, 'eval_mar_100_top, t-shirt, sweatshirt': 0.4731, 'eval_map_sweater': 0.0, 'eval_mar_100_sweater': 0.0, 'eval_map_cardigan': 0.0, 'eval_mar_100_cardigan': 0.0, 'eval_map_jacket': 0.1701, 'eval_mar_100_jacket': 0.6348, 'eval_map_vest': 0.0, 'eval_mar_100_vest': 0.0, 'eval_map_pants': 0.3809, 'eval_mar_100_pants': 0.677, 'eval_map_shorts': 0.0353, 'eval_mar_100_shorts': 0.0904, 'eval_map_skirt': 0.0765, 'eval_mar_100_skirt': 0.4514, 'eval_map_coat': 0.0, 'eval_mar_100_coat': 0.0, 'eval_map_dress': 0.3374, 'eval_mar_

In [28]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:


# Evaluate the model on the test dataset
# metrics = trainer.evaluate(test_dataset_transformed)
batch_size = trainer.args.per_device_eval_batch_size
print(batch_size)
# Extract required values
selected_metrics = {
    "Precision (mAP)": metrics.get("eval_map", 0),
    "Recall (mAR_100)": metrics.get("eval_mar_100", 0),
    "Inference Time (s)": 1 / (metrics.get("eval_steps_per_second", 1) * batch_size)
}

# Create a PrettyTable for display
table = PrettyTable()
table.field_names = ["Metric", "Value"]

# Add metric results to the table
for key, value in selected_metrics.items():
    table.add_row([key, round(value, 4)])

# Print the formatted table
print(table)

4
+--------------------+--------+
|       Metric       | Value  |
+--------------------+--------+
|  Precision (mAP)   | 0.0436 |
|  Recall (mAR_100)  | 0.1315 |
| Inference Time (s) | 0.047  |
+--------------------+--------+


# Testing on single image

In [ ]:


idx=50 # you can change these image ids to test on different images from test dataset [0, 1000]
sample = test_dataset[idx]
image = sample["image"]
image_bytes = image["bytes"]  # Extract bytes from dictionary
image = Image.open(io.BytesIO(image_bytes))

#loading fine-tuned model for testing
obj_detector = pipeline(
    "object-detection", model="/home/utn/firi22ka/Desktop/jenga/mlp/g10/test/detr-resnet-50-fashionpedia-full-finetuned_5epochs/checkpoint-16300"  # Change with your model name
)


results = obj_detector(image)
print(results)

In [ ]:

def plot_results(image, results, threshold=0.6):
    image = Image.fromarray(np.uint8(image))
    draw = ImageDraw.Draw(image)
    width, height = image.size

    for result in results:
        score = result["score"]
        label = result["label"]
        box = list(result["box"].values())

        if score > threshold:
            x1, y1, x2, y2 = tuple(box)
            draw.rectangle((x1, y1, x2, y2), outline="red", width=3)
            draw.text((x1 + 5, y1 - 10), label, fill="white")
            draw.text((x1 + 5, y1 + 10), f"{score:.2f}", fill="green" if score > 0.7 else "red")

    return image

In [ ]:
plot_results(image, results)

# To visualize and save results on mutilple images a

In [ ]:


def visualy_compare_object_detection(dataset, output_dir= 'result/detr', num_samples=4):
    os.makedirs(output_dir, exist_ok=True)
    indices = random.sample(range(len(test_dataset)), num_samples)
    print(f'visualizing performance of fine-tuned models on {dataset} images indexed at {indices}')
    # Load the pretrained model
    obj_detector_pretrained = pipeline(
        "object-detection", model="facebook/detr-resnet-50"  
    )

    # Load the fine-tuned model
    obj_detector_finetuned = pipeline(
        "object-detection", model="/home/utn/firi22ka/Desktop/jenga/mlp/g10/test/detr-resnet-50-fashionpedia-full-finetuned_5epochs/checkpoint-25000" 
    )
    for idx in indices:
        sample = dataset[idx]
        image = sample["image"]
        image_bytes = image["bytes"]  # Extract bytes from dictionary
        image = Image.open(io.BytesIO(image_bytes))
        image_np = np.array(image)

        # Object detection using pretrained model
        results_pretrained = obj_detector_pretrained(image)
        
        # # Object detection  using fine-tuned model
        results_finetuned = obj_detector_finetuned(image)

        # Plotting & Saving ground truth images
        ground_truth_with_boxes = draw_image_from_idx(dataset, idx)
        ground_truth_with_boxes.save(f"{output_dir}/{idx}_gt.jpg")

        # Plotting & Saving results for the pretrained model
        image_with_pretrained_detections = plot_results(image_np, results_pretrained)
        image_with_pretrained_detections.save(f"{output_dir}/{idx}_pretrained.jpg")

        # # Plotting & Saving results for the fine-tuned model
        image_with_finetuned_detections = plot_results(image_np, results_finetuned)
        image_with_finetuned_detections.save(f"{output_dir}/{idx}_finetuned.jpg")

        print(f"Results for Image {idx} are saved at {output_dir}")
        


visualy_compare_object_detection(dataset=test_dataset, output_dir= "resutl/detr", num_samples= 4) # change number of sample to check result on tham many images